In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


In [2]:
# Veri setini yükleyelim
url = 'https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv'
data = pd.read_csv(url)
data.head()

In [3]:
# Eksik değerlerin kontrolü
data.isnull().sum()

In [4]:
# Outlier'ları görselleştirme
plt.figure(figsize=(12, 6))
sns.boxplot(data=data['medv'])  # medv (median value) bağımlı değişken
plt.title('Outliers in Target Variable')
plt.show()

In [5]:
# Outlier'ları kaldırma
Q1 = data['medv'].quantile(0.25)
Q3 = data['medv'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
data = data[(data['medv'] > lower_bound) & (data['medv'] < upper_bound)]
data.head()

In [6]:
# Özellikler ve bağımlı değişken
X = data.drop(columns=['medv'])  # Bağımlı değişkeni çıkarıyoruz
y = data['medv']
X.head()

In [7]:
# Veriyi eğitim, doğrulama ve test setlerine bölüyoruz
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train.shape, X_val.shape, X_test.shape

In [8]:
# Modelleri eğitelim
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42)
}
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    results[name] = rmse
results

In [9]:
# En iyi modeli seçelim
best_model_name = min(results, key=results.get)
best_model_name, results[best_model_name]

In [10]:
# Test seti üzerindeki başarıyı kontrol edelim
best_model = models[best_model_name]
test_preds = best_model.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, test_preds))
test_rmse

In [11]:
# Özelliklerin önem derecelerini hesaplayalım
feature_importances = best_model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)
importance_df

In [12]:
# Özelliklerin önem derecelerini görselleştirelim
importance_df.plot(kind='bar', x='Feature', y='Importance', title='Feature Importances', legend=False)
plt.show()

# Sonuçlar
### En İyi Model: Random Forest
En iyi model **Random Forest Regressor** olup, modelin doğrulama setindeki **RMSE (Root Mean Squared Error)** değeri **2.93**'tür.
### Test Sonuçları
Test seti üzerindeki **RMSE** değeri ise **2.43**'tür.
### Özellik Önem Dereceleri
Random Forest modelinin özellikler için hesapladığı önem dereceleri aşağıdaki tabloda verilmiştir:
![Feature Importances](images/feature_importances.png)
### Outlier Görselleştirmesi
![Outliers in Target Variable](images/outliers.png)